In [35]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Rescaling, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load the data
data_dir = 'C:/test001/ttt/data'
batch_size = 32
img_height = 28
img_width = 28

train_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)

# Data augmentation
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal_and_vertical"),
    keras.layers.RandomRotation(0.2),
])

# Configure the dataset for performance
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Create the model
num_classes = len(class_names)

model = keras.Sequential([
    data_augmentation,
    Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes)
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

model.summary()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

# Train the model
epochs = 30
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr]
)

# Save the model as a TFLite file
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('C:/test001/ttt/cnn_classifier/ml_model/model.tflite', 'wb') as f:
    f.write(tflite_model)


Found 79895 files belonging to 8 classes.
Using 63916 files for training.
Found 79895 files belonging to 8 classes.
Using 15979 files for validation.
['airplane', 'alarm_clock', 'apple', 'basketball', 'bat', 'bathtub', 'book', 'cycle']


c:\test001\ttt\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
1998/1998 ━━━━━━━━━━━━━━━━━━━━ 23s 8ms/step - accuracy: 0.5739 - loss: 1.2106 - val_accuracy: 0.8096 - val_loss: 0.6075 - learning_rate: 0.0010
Epoch 2/30
1998/1998 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.7944 - loss: 0.6344 - val_accuracy: 0.8445 - val_loss: 0.5058 - learning_rate: 0.0010
Epoch 3/30
1998/1998 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8164 - loss: 0.5606 - val_accuracy: 0.8561 - val_loss: 0.4884 - learning_rate: 0.0010
Epoch 4/30
1998/1998 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8355 - loss: 0.5091 - val_accuracy: 0.8765 - val_loss: 0.4053 - learning_rate: 0.0010
Epoch 5/30
1998/1998 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8458 - loss: 0.4776 - val_accuracy: 0.8825 - val_loss: 0.3998 - learning_rate: 0.0010
Epoch 6/30
1998/1998 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8518 - loss: 0.4608 - val_accuracy: 0.8899 - val_loss: 0.3583 - learning_rate: 0.0010
Epoch 7/30
1998/1998 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.8

INFO:tensorflow:Assets written to: C:\Users\Mind\AppData\Local\Temp\tmpfjlyc720\assets


Saved artifact at 'C:\Users\Mind\AppData\Local\Temp\tmpfjlyc720'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 3), dtype=tf.float32, name='keras_tensor_220')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  2231154712912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2231154717712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2231154718864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2231154709648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2231154719248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2231154718480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2231154718288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2232244289168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2232243126928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2232243127312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2232

In [62]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 28, 28, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 28, 28, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 13, 13, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 5, 5, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1, 1, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 334,170 (1.27 MB)

 Trainable params: 111,240 (434.53 KB)

 Non-trainable params: 448 (1.75 KB)

 Optimizer params: 222,482 (869.07 KB)

In [44]:
# Save the model as a TFLite file
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model saved as model.tflite")


INFO:tensorflow:Assets written to: C:\Users\Mind\AppData\Local\Temp\tmpo8kzgyk8\assets


INFO:tensorflow:Assets written to: C:\Users\Mind\AppData\Local\Temp\tmpo8kzgyk8\assets


Saved artifact at 'C:\Users\Mind\AppData\Local\Temp\tmpo8kzgyk8'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 3), dtype=tf.float32, name='keras_tensor_220')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  2231154712912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2231154717712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2231154718864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2231154709648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2231154719248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2231154718480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2231154718288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2232244289168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2232243126928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2232243127312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2232

In [73]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

class_names = ['airplane',
 'alarm_clock',
 'apple',
 'basketball',
 'bat',
 'bathtub',
 'book',
 'cycle']

# Load the TFLite model and allocate tensors
model_path = 'C:/test001/ttt/cnn_classifier/ml_model/ios_model.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Define the image path
img_path = 'C:/test001/ttt/cnn_classifier/iamges/image_4.png'

# Load and preprocess the image
img_height = 28
img_width = 28

def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Create a batch axis
    img_array = img_array # Normalize the image
    return img_array

img_array = load_and_preprocess_image(img_path)

# Set the tensor to point to the input data to be inferred
interpreter.set_tensor(input_details[0]['index'], img_array)

# Run the inference
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])
predicted_class = np.argmax(output_data, axis=1)
class_names[predicted_class[0]]

'bat'

In [74]:
output_data

array([[ -1.2884495,  -6.8941965,  -6.5475106, -11.686626 ,  17.954012 ,
         -6.8227386,  -8.477249 ,  -7.1611204]], dtype=float32)

In [69]:
input_details

[{'name': 'serving_default_keras_tensor_220:0',
  'index': 0,
  'shape': array([ 1, 28, 28,  3]),
  'shape_signature': array([-1, 28, 28,  3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [66]:
class_names1 = ['airplane',
 'alarm_clock',
 'apple',
 'basketball',
 'bat',
 'bathtub',
 'book',
 'cycle']

In [77]:
import tensorflow as tf
import numpy as np

def print_tflite_model_details(model_path):
    # Load the TFLite model
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    # Get input and output tensors details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    print("Model Architecture:")
    print("-------------------")
    print("Inputs:")
    for input_tensor in input_details:
        print(f"Name: {input_tensor['name']}")
        print(f"Shape: {input_tensor['shape']}")
        print(f"Type: {input_tensor['dtype']}")
        print()

    print("Outputs:")
    for output_tensor in output_details:
        print(f"Name: {output_tensor['name']}")
        print(f"Shape: {output_tensor['shape']}")
        print(f"Type: {output_tensor['dtype']}")
        print()

    print("Layers:")
    # Iterate through all tensors and extract layer information
    for i in range(len(interpreter.get_tensor_details())):
        tensor = interpreter.get_tensor_details()[i]
        print(f"Layer {i+1}:")
        print(f"Name: {tensor['name']}")
        print(f"Index: {tensor['index']}")
        print(f"Shape: {tensor['shape']}")
        print(f"Type: {tensor['dtype']}")
        print(f"Quantization: {tensor['quantization']}")
        print("-------------------")

# Path to your TFLite model
model_path = 'C:/test001/ttt/cnn_classifier/ml_model/ios_model.tflite'

# Print the model details
print_tflite_model_details(model_path)


Model Architecture:
-------------------
Inputs:
Name: serving_default_keras_tensor_220:0
Shape: [ 1 28 28  3]
Type: <class 'numpy.float32'>

Outputs:
Name: StatefulPartitionedCall_1:0
Shape: [1 8]
Type: <class 'numpy.float32'>

Layers:
Layer 1:
Name: serving_default_keras_tensor_220:0
Index: 0
Shape: [ 1 28 28  3]
Type: <class 'numpy.float32'>
Quantization: (0.0, 0)
-------------------
Layer 2:
Name: arith.constant
Index: 1
Shape: [128]
Type: <class 'numpy.float32'>
Quantization: (0.0, 0)
-------------------
Layer 3:
Name: arith.constant1
Index: 2
Shape: [64]
Type: <class 'numpy.float32'>
Quantization: (0.0, 0)
-------------------
Layer 4:
Name: arith.constant2
Index: 3
Shape: [32]
Type: <class 'numpy.float32'>
Quantization: (0.0, 0)
-------------------
Layer 5:
Name: arith.constant3
Index: 4
Shape: [128   3   3  64]
Type: <class 'numpy.float32'>
Quantization: (0.0, 0)
-------------------
Layer 6:
Name: arith.constant4
Index: 5
Shape: [64  3  3 32]
Type: <class 'numpy.float32'>
Quantiz

In [88]:
output_details

[{'name': 'StatefulPartitionedCall_1:0',
  'index': 32,
  'shape': array([1, 8]),
  'shape_signature': array([-1,  8]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [ ]:
from tflite_support.metadata_writers import image_classifier
from tflite_support.metadata_writers import writer_utils

# Paths to the model and label file
model_path = 'C:/test001/ttt/cnn_classifier/ml_model/ios_model.tflite'
label_file = 'C:/test001/ttt/cnn_classifier/ml_model/labels.txt'
model_with_metadata_path = 'C:/test001/ttt/cnn_classifier/ml_model/ios_model_with_metadata.tflite'

# Create a writer
writer = image_classifier.MetadataWriter.create_for_inference(
    writer_utils.load_file(model_path),
    input_norm_mean=[127.5],
    input_norm_std=[127.5],
    label_file_paths=[label_file]
)

# Verify the metadata
print(writer.get_metadata_json())

# Write the metadata to the model file
writer_utils.save_file(writer.populate(), model_with_metadata_path)


In [85]:
from tflite_support.metadata_writers import image_classifier
from tflite_support.metadata_writers import writer_utils
from tflite_support import metadata

# Paths to your model and the new model with metadata
model_path = 'C:/test001/ttt/cnn_classifier/ml_model/model.tflite'
output_model_path = 'C:/test001/ttt/cnn_classifier/ml_model/model_with_metadata.tflite'




# Create the metadata writer
writer = image_classifier.MetadataWriter.create_for_inference(
    model_path=model_path,
    label_file_paths=['C:/test001/ttt/cnn_classifier/notebook/label.txt'],
    input_norm_mean=[0],  # Adjust these values if your model requires different normalization
    input_norm_std=[255])

# Write the metadata to the model
writer_utils.save_file(writer.populate(), output_model_path)

print("Metadata added and saved to", output_model_path)


ModuleNotFoundError: No module named 'tflite_support'

In [83]:
!pip install --use-pep517 tflite-support


  Using cached tflite-support-0.1.0a1.tar.gz (390 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached pybind11-2.13.1-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.1-py3-none-any.whl (238 kB)
Failed to build tflite-support


  error: subprocess-exited-with-error
  
  × Building wheel for tflite-support (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [74 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-312
      creating build\lib.win-amd64-cpython-312\tflite_support
      copying tflite_support\codegen.py -> build\lib.win-amd64-cpython-312\tflite_support
      copying tflite_support\metadata.py -> build\lib.win-amd64-cpython-312\tflite_support
      copying tflite_support\metadata_schema_py_generated.py -> build\lib.win-amd64-cpython-312\tflite_support
      copying tflite_support\schema_py_generated.py -> build\lib.win-amd64-cpython-312\tflite_support
      copying tflite_support\__init__.py -> build\lib.win-amd64-cpython-312\tflite_support
      running egg_info
      writing tflite_support.egg-info\PKG-INFO
      writing dependency_links to tflite_support.egg-info\dependency_links.txt
 

In [84]:
from tflite_support import metadata 

ModuleNotFoundError: No module named 'tflite_support'

In [89]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [90]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPU devices:", tf.config.list_physical_devices('GPU'))


TensorFlow version: 2.17.0
Built with CUDA: False
GPU devices: []


In [1]:
import tensorflow as tf


# Check for GPU availability
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Set memory growth for GPUs
tf.config.experimental.list_physical_devices('GPU')

Num GPUs Available:  0


[]

In [2]:
import tensorflow as tf

# Load the TFLite model and allocate tensors.
model_path = 'C:/test001/ttt/cnn_classifier/ml_model/ios_model.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input details.
input_details = interpreter.get_input_details()

# Print input details.
print("Input details:")
print(input_details)

# Extract and print the input shape.
input_shape = input_details[0]['shape']
print("Expected input shape:", input_shape)


Input details:
[{'name': 'serving_default_keras_tensor_220:0', 'index': 0, 'shape': array([ 1, 28, 28,  3]), 'shape_signature': array([-1, 28, 28,  3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Expected input shape: [ 1 28 28  3]


In [3]:
import tensorflow as tf

# Load the TFLite model and allocate tensors.
model_path = 'C:/test001/ttt/cnn_classifier/ml_model/model345.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input details.
input_details = interpreter.get_input_details()

# Print input details.
print("Input details:")
print(input_details)

# Extract and print the input shape.
input_shape = input_details[0]['shape']
print("Expected input shape:", input_shape)

Input details:
[{'name': 'input_1', 'index': 15, 'shape': array([ 1, 28, 28,  1]), 'shape_signature': array([ 1, 28, 28,  1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Expected input shape: [ 1 28 28  1]
